Import public packages and functions

In [1]:
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import compress
from itertools import product

import openpyxl
from datetime import datetime
from scipy.spatial import distance
from scipy.ndimage import uniform_filter1d
import math
from scipy.signal import find_peaks, peak_prominences
import statistics as stat
import json

In [2]:
def get_repo_path_in_notebook():
    """
    Finds path of repo from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()
    repo_name = 'ultraleap_analysis'

    while path[-len(repo_name):] != 'ultraleap_analysis':

        path = os.path.dirname(path)

    return path

In [3]:
repo_path = get_repo_path_in_notebook()
code_path = os.path.join(repo_path, 'code')
os.chdir(code_path)


Import own functions

In [4]:
import import_data.import_and_convert_data as import_dat
import import_data.find_paths as find_paths
import import_data.preprocessing_meta_info as meta_info
import sig_processing.segment_tasks as tasks
import movement_calc.helpfunctions as hp
import sig_processing.calc_features as feat_ext


### Saving blocks for preprocessing

Define variables of interest

In [5]:
folder = 'patientdata'
ls_cam = ['vr', 'dt', 'st']
ls_task = ['ps','pt']
ls_cond = ['m0s0', 'm1s0', 'm1s1', 'm0s1', 'm1', 'm0']
ls_side = ['left', 'right']

Save blocks

In [6]:
subs = find_paths.find_available_subs(folder)

In [7]:
# Reloading own functions
importlib.reload(import_dat)
importlib.reload(tasks)
importlib.reload(find_paths)
importlib.reload(feat_ext)

for sub in subs:
    for task in ls_task:
        for side in ls_side:
            for cam in ls_cam:
                for cond in ls_cond:
                    try:
                        # get defined data
                        data = import_dat.get_data(
                            folder=folder, sub=sub, cam_pos=cam,
                            task=task, condition=cond, side=side,
                        )

                        # get block dataframes
                        blocks = tasks.block_extraction(
                            data, sub=sub, task=task, side=side,
                            cond=cond, cam=cam,
                            to_save=True, to_plot = True
                        )

                    except:
                        print('\n')
                        # print(f'Combination does not exist: ' f'{sub, cam, task, cond, side}')


No files available for combination: ('ul001', 'vr', 'ps', 'm0s0', 'left')


No files available for combination: ('ul001', 'vr', 'ps', 'm1s0', 'left')


No files available for combination: ('ul001', 'vr', 'ps', 'm1s1', 'left')


No files available for combination: ('ul001', 'vr', 'ps', 'm0s1', 'left')




No files available for combination: ('ul001', 'vr', 'ps', 'm0', 'left')


No files available for combination: ('ul001', 'dt', 'ps', 'm0s0', 'left')


No files available for combination: ('ul001', 'dt', 'ps', 'm1s0', 'left')


No files available for combination: ('ul001', 'dt', 'ps', 'm1s1', 'left')


No files available for combination: ('ul001', 'dt', 'ps', 'm0s1', 'left')




No files available for combination: ('ul001', 'dt', 'ps', 'm0', 'left')














No files available for combination: ('ul001', 'vr', 'ps', 'm0s0', 'right')


No files available for combination: ('ul001', 'vr', 'ps', 'm1s0', 'right')


No files available for combination: ('ul001', 'vr', 'ps', 'm1s1', 'right')

Load blocks, calculate distances and create dist-time .csv files

In [7]:
subs = find_paths.find_available_subs(folder)
ls_task = ['ft', 'oc']
ls_cond = ['m0s0', 'm1s0', 'm1s1', 'm0s1', 'm1', 'm0']

In [12]:
# Reloading own functions
importlib.reload(import_dat)
importlib.reload(tasks)
importlib.reload(find_paths)
importlib.reload(feat_ext)

for sub in subs:
    for task in ls_task:
        for cond in ls_cond:
            try:
                files = os.listdir(os.path.join(repo_path, 'data', sub, task, cond, 'cleaned_blocks'))

                for file in files:
                    # print(file)
                    # Load blocks from patients' blocks dir
                    block = pd.read_csv(os.path.join(
                        repo_path, 'data', sub, task, cond, 'cleaned_blocks', file), index_col=0)

                    if task == 'oc':
                        ls_fing = ['index_tip', 'middle_tip', 'ring_tip', 'pinky_tip']
                        dist = {}
                        for fing in ls_fing:
                            dist[fing] = []
                        for fing in ls_fing:
                            dist[fing] = hp.calc_distances(block, fing, 'palm')
                        
                        dist_time = pd.DataFrame(dist, columns=ls_fing)
                        
                        # insert mean column
                        mean = np.mean(dist_time[ls_fing], axis=1)
                        dist_time['mean'] = mean
                        dist_time['time'] = block.program_time.to_list()

                        ref_time = [0]
                        cum_time = 0
                        for time in np.diff(block.program_time):
                            cum_time += time
                            ref_time.append(cum_time)
                        dist_time['ref_time'] = ref_time

                        # dist_time['global_time'] = block.global_time.to_list()
                        dist_time['date_time'] = block.date_time.to_list()

                    elif task == 'ft':
                        dist = hp.calc_distances(block, 'index_tip', 'thumb_tip')
                        dist_time = pd.DataFrame(dist, columns=['dist'])
                        dist_time['time'] = block.program_time.to_list()
                        
                        ref_time = [0]
                        cum_time = 0
                        for time in np.diff(block.program_time):
                            cum_time += time
                            ref_time.append(cum_time)
                        dist_time['ref_time'] = ref_time

                        # dist_time['global_time'] = block.global_time.to_list()
                        dist_time['date_time'] = block.date_time.to_list()
                        

                    elif task == 'ps':
                        dist = hp.PS_ang(block, 'thumb_tip', 'middle_tip', 'palm')
                        dist_time = pd.DataFrame(dist, columns=['ang'])
                        dist_time['time'] = block.program_time.to_list()

                        ref_time = [0]
                        cum_time = 0
                        for time in np.diff(block.program_time):
                            cum_time += time
                            ref_time.append(cum_time)
                        dist_time['ref_time'] = ref_time

                        # dist_time['global_time'] = block.global_time.to_list()
                        dist_time['date_time'] = block.date_time.to_list()

                    else:
                        print('postural tremor was not analysed yet')

                    dist_time_path = os.path.join(
                        repo_path, 'data', sub, task, cond, 'distances')
                    if not os.path.exists(dist_time_path):
                        os.makedirs(dist_time_path)
                    dist_time.to_csv(os.path.join(
                        dist_time_path, f'{file}'))
                    
            except FileNotFoundError:
                print('No files available for combination: '
                    f'{sub, task, cond}')

No files available for combination: ('ul020', 'ft', 'm0s0')
No files available for combination: ('ul020', 'ft', 'm1s0')
No files available for combination: ('ul020', 'ft', 'm0s1')
No files available for combination: ('ul020', 'ft', 'm1')
No files available for combination: ('ul020', 'ft', 'm0')
No files available for combination: ('ul020', 'oc', 'm1s0')
No files available for combination: ('ul020', 'oc', 'm0s1')
No files available for combination: ('ul020', 'oc', 'm1')
No files available for combination: ('ul020', 'oc', 'm0')
